In [1]:
# Standard libraries
import h5py
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive

# Append base directory
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
path1p = os.path.dirname(currentdir)
path2p = os.path.dirname(path1p)
libpath = os.path.join(path1p, "lib")
pwd_mat = os.path.join(os.path.join(path2p, "data/"), "sim_ds_mat")
pwd_h5 = os.path.join(os.path.join(path2p, "data/"), "sim_ds_h5")

sys.path.insert(0, libpath)
print("Appended library directory", libpath)

# User libraries
from matlab.matlab_yaro_lib import read_mat
from signal_lib import downsample
from corr_lib import sprMat
from idtxl_wrapper import idtxlParallelCPUMulti, idtxlResultsParse
from qt_wrapper import gui_fpath

# IDTxl libraries
from idtxl.bivariate_mi import BivariateMI
from idtxl.multivariate_mi import MultivariateMI
from idtxl.bivariate_te import BivariateTE
from idtxl.multivariate_te import MultivariateTE
from idtxl.data import Data
from idtxl.visualise_graph import plot_network

Appended library directory /home/ubuntu/work/nest-mesoscopic-metric-test/codes/lib


## 1. Load Neuronal Data

The neuronal data for a given experiment is given by the 3D matrix of dimensions `[N_Trial, N_Times, N_Channels]`

Notes:
* `DT = 0.05` : the sampling interval is always 50ms
* `N_Times = 201` : the total experiment time is always 10s. All experiments are aligned

In [2]:
# Declare constants
params = {
    "exp_timestep" : 0.05, # 50ms, the standard measurement interval
    "bin_timestep" : 0.2,  # 200ms, the binned interval
    
    # The standard timesteps for different scenarios
    "range_CUE" : (1.0, 1.5),    # 1-1.5 seconds trial time
    "range_TEX" : (2.0, 3.5),    # 2-3.5 seconds trial time
    "range_LIK" : (3.5, 6.0),    # 3.5-6 seconds trial time
    "range_ALL" : (0.0, 10.0),   # 0-10  seconds

#     "samples_window" : "ALL",
#     "trials_type"    : "GO",
    "subsampling"    : True,   # To bin or not to bin data
#     "subsampling_windows_lenghts" : 4*np.ones((1,51)),  #uniform subsampling, 50 bins
    
    "te_delay_min" : 1,
    "te_delay_max" : 5,
    
#     "te_delay_stepsize" : 1,
#     "te_receiver_embedding_tau" : 1,
#     "te_sampling_rate" : 20
}

# GUI: Select videos for training
pwd_tmp = "./"
next_path = "AAAAA"
params['pwd_data_folders'] = []
while next_path != '':
    print("Currently using data folders:", params['pwd_data_folders'])
    next_path = gui_fpath("Select one more data directory, or cancel", pwd_tmp)
    if next_path != '':
        params['pwd_data_folders'] += [next_path]
        pwd_tmp = os.path.dirname(next_path)  # Next time choose from parent folder
print('Done')

Currently using data folders: []
Currently using data folders: ['/mnt/hifo-scratch-01/Yaro/data_for_TE/mtp/mtp_1/mtp_1_2015_06_18_b']
Done


## Swipe Transfer Entropy via IDTxl

Notation of IDTxl
* `(processes, samples, replications)` equivalent to ours `(channels, times, trials)`
* Thus Yaro matrix `(nTrials, nTimes, nChannels)` is given by `'rsp'`, but
* For example `(nChannels, nTimes)` would be `'ps'`

In [3]:
%%time
for folderPathName in params['pwd_data_folders']:
    folderName = os.path.basename(folderPathName)
    
    #############################
    # Reading and parsing
    #############################

    # Read LVM file from command line
    data, behaviour = read_mat(folderPathName)

    # Get parameters
    nTrials, nTimes, nChannels = data.shape
    print("Loaded neuronal data with (nTrials, nTimes, nChannels)=", data.shape)

    assert nTimes == 201, "The number of timesteps must be 201 for all data, got "+str(nTimes)
    nTimesDownsampled = int(nTimes * params["exp_timestep"] / params["bin_timestep"])
    print(nTimesDownsampled)

    # Timeline (x-axis)
    tlst = params["exp_timestep"] * np.linspace(0, nTimes, nTimes)

    # Downsample data
    print("Downsampling from", params["exp_timestep"], "ms to", params["bin_timestep"], "ms")
    if params["subsampling"]:
        params["timestep"] = params["bin_timestep"]
        tlst_new = downsample(tlst, data[0, :, 0], nTimesDownsampled)[0]
        data_new = np.array([[downsample(tlst, data[i, :, j], nTimesDownsampled)[1] 
                              for i in range(nTrials)] 
                             for j in range(nChannels)])
    else:
        params["timestep"] = params["exp_timestep"]

#     # Plot downsampling for comparison
#     plt.figure()
#     plt.plot(tlst, data[0, :, 0], label='original')
#     plt.plot(tlst_new, data_new[0, 0, :], '.-', label='downsampled')
#     plt.title("Example before and after downsampling")
#     plt.legend()
#     plt.show()

    # Replace old data with subsampled one    
    tlst, data = tlst_new, data_new.transpose(1, 2, 0)
    nTrials, nTimes, nChannels = data.shape
    print("After downsampling data shape is (nTrials, nTimes, nChannels)=", data.shape)
    
    #############################
    # Analysis
    #############################
    
    idtxl_settings = {
        'dim_order'       : 'rsp',
        'methods'         : ["BivariateTE", "MultivariateTE"],
        'cmi_estimator'   : 'JidtGaussianCMI',
        'max_lag_sources' : params["te_delay_max"],
        'min_lag_sources' : params["te_delay_min"]
   }

    teWindow = params["te_delay_max"] + 1

    teSwipeMat  = np.zeros((nChannels, nChannels, nTimes)) + np.nan
    lagSwipeMat = np.zeros((nChannels, nChannels, nTimes)) + np.nan
    pSwipeMat   = np.zeros((nChannels, nChannels, nTimes)) + np.nan
    
    data_range = list(range(nTimes - teWindow))
    data_lst = [data[:, i:i + teWindow, :] for i in data_range]
    rez = idtxlParallelCPUMulti(data_lst, idtxl_settings, folderName)
    
    for iMethod, method in enumerate(idtxl_settings['methods']):
        for iRange in data_range:
            te_mat, lag_mat, p_mat = idtxlResultsParse(rez[iMethod][iRange], nChannels, method=method, storage='matrix')

            teSwipeMat[:,:, iRange + params["te_delay_max"]]  = te_mat
            lagSwipeMat[:,:, iRange + params["te_delay_max"]] = lag_mat
            pSwipeMat[:,:, iRange + params["te_delay_max"]]   = p_mat

        #######################
        # Save results to file
        #######################
        savename = os.path.join(pwd_h5, folderName + '_' + method + '_swipe' + '.h5')
        print(savename)

        h5f = h5py.File(savename, "w")

        grp_rez = h5f.create_group("results")
        grp_rez['TE_table'] = te_mat
        grp_rez['delay_table'] = lag_mat
        grp_rez['p_table'] = p_mat

        h5f.close()

Reading Yaro data from /mnt/hifo-scratch-01/Yaro/data_for_TE/mtp/mtp_1/mtp_1_2015_06_18_b
Loaded neuronal data with (nTrials, nTimes, nChannels)= (198, 201, 12)
50
Downsampling from 0.05 ms to 0.2 ms
After downsampling data shape is (nTrials, nTimes, nChannels)= (198, 50, 12)
Adding data with properties: 12 processes, 6 samples, 198 replications
overwriting existing data
Adding data with properties: 12 processes, 6 samples, 198 replications
overwriting existing data
Adding data with properties: 12 processes, 6 samples, 198 replications
overwriting existing data
Adding data with properties: 12 processes, 6 samples, 198 replications
overwriting existing data
Adding data with properties: 12 processes, 6 samples, 198 replications
overwriting existing data
Adding data with properties: 12 processes, 6 samples, 198 replications
overwriting existing data
Adding data with properties: 12 processes, 6 samples, 198 replications
overwriting existing data
Adding data with properties: 12 processes, 6

testing candidate: (9, 1) maximum statistic, n_perm: 200
testing candidate: (3, 1) maximum statistic, n_perm: 200

Target: 0 - testing sources [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Target: 9 - testing sources [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11]


---------------------------- (1) include target candidates
---------------------------- (1) include target candidates

candidate set: [(9, 1), (9, 2), (9, 3), (9, 4), (9, 5)]
candidate set: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]
testing candidate: (0, 1) maximum statistic, n_perm: 200

Target: 3 - testing sources [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11]

---------------------------- (1) include target candidates
candidate set: [(3, 1), (3, 2), (3, 3), (3, 4), (3, 5)]
testing candidate: (9, 1) maximum statistic, n_perm: 200
testing candidate: (3, 1) maximum statistic, n_perm: 200

Target: 6 - testing sources [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11]

---------------------------- (1) include target candidates
Target: 0 - testing sources [1, 2, 3, 4, 5, 6, 

testing candidate: (6, 4) maximum statistic, n_perm: 200
testing candidate: (3, 4) maximum statistic, n_perm: 200
testing candidate: (3, 4) maximum statistic, n_perm: 200
testing candidate: (9, 3) maximum statistic, n_perm: 200
testing candidate: (6, 4) maximum statistic, n_perm: 200
testing candidate: (1, 2) maximum statistic, n_perm: 200
testing candidate: (0, 4) maximum statistic, n_perm: 200
testing candidate: (1, 2) maximum statistic, n_perm: 200
 -- not significant
 -- not significant

---------------------------- (2) include source candidates
---------------------------- (2) include source candidates
candidate set current source: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]

candidate set current source: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]
testing candidate: (0, 1) maximum statistic, n_perm: 200
testing candidate: (0, 1) maximum statistic, n_perm: 200
 -- not significant
testing candidate: (0, 2) maximum statistic, n_perm: 200
testing candidate: (0, 3) maximum statistic, n_perm

testing candidate: (2, 3) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)]
testing candidate: (1, 1) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(2, 1), (2, 2), (2, 3), (2, 4), (2, 5)]
testing candidate: (2, 3) maximum statistic, n_perm: 200
testing candidate: (0, 5) maximum statistic, n_perm: 200
testing candidate: (0, 4) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)]
testing candidate: (1, 5) maximum statistic, n_perm: 200
testing candidate: (0, 5) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)]
testing candidate: (1, 1) maximum statistic, n_perm: 200
testing candidate: (0, 2) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)]
testing candidate: (1, 1) maximu

testing candidate: (1, 5) maximum statistic, n_perm: 200
testing candidate: (3, 3) maximum statistic, n_perm: 200
 -- not significant
testing candidate: (1, 5) maximum statistic, n_perm: 200
candidate set current source: [(2, 1), (2, 2), (2, 3), (2, 4), (2, 5)]
testing candidate: (2, 1) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(2, 1), (2, 2), (2, 3), (2, 4), (2, 5)]
testing candidate: (2, 1) maximum statistic, n_perm: 200
testing candidate: (2, 5) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(4, 1), (4, 2), (4, 3), (4, 4), (4, 5)]
testing candidate: (4, 1) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(3, 1), (3, 2), (3, 3), (3, 4), (3, 5)]
testing candidate: (3, 1) maximum statistic, n_perm: 200
testing candidate: (1, 4) maximum statistic, n_perm: 200
testing candidate: (2, 5) maximum statistic, n_perm: 200
testing candidate: (2, 2) maximum statistic, n_perm: 200
 -- not s

...

testing candidate: (1, 5) maximum statistic, n_perm: 200
testing candidate: (3, 1) maximum statistic, n_perm: 200
 -- not significant

---------------------------- (3) prune source candidate
selected candidates: [(4, 1)]
 -- significant

---------------------------- (4) final statistics
selected variables: [(0, 1), (0, 2), (4, 1)]
omnibus test, n_perm: 500
testing candidate: (0, 1) maximum statistic, n_perm: 200
 -- significant

sequential maximum statistic, n_perm: 500
testing candidate: (9, 2) maximum statistic, n_perm: 200
final source samples: [(4, 1)]
final target samples: [(0, 1), (0, 2)]



Target: 1 - testing sources [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

---------------------------- (1) include target candidates
candidate set: [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)]
testing candidate: (1, 1) maximum statistic, n_perm: 200
 -- not significant

---------------------------- (3) prune source candidate
selected candidates: [(8, 1), (8, 4)]
testing candidate: (8, 4) minimum statistic

testing candidate: (4, 1) maximum statistic, n_perm: 200
testing candidate: (2, 4) maximum statistic, n_perm: 200
 -- not significant

---------------------------- (3) prune source candidate
selected candidates: [(8, 1), (0, 1), (8, 3), (9, 1), (9, 2)]
testing candidate: (8, 3) minimum statistic, n_perm: 500
testing candidate: (2, 2) maximum statistic, n_perm: 200
testing candidate: (2, 5) maximum statistic, n_perm: 200
 -- not significant

---------------------------- (2) include source candidates
candidate set: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (10, 1), (10, 2), (10, 3), (10, 4), (10, 5), (11, 1), (11, 2), (11, 3), (11, 4), (11, 5)]
testing candidate: (4, 1) maxi

testing candidate: (11, 3) maximum statistic, n_perm: 200
testing candidate: (11, 5) maximum statistic, n_perm: 200
 -- not significant

---------------------------- (2) include source candidates
candidate set: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (10, 1), (10, 2), (10, 3), (10, 4), (10, 5)]
testing candidate: (7, 1) maximum statistic, n_perm: 200
 -- significant

---------------------------- (4) final statistics
selected variables: [(8, 1), (6, 1), (9, 1), (9, 2)]
omnibus test, n_perm: 500
 -- significant

sequential maximum statistic, n_perm: 500
final source samples: [(6, 1), (9, 1), (9, 2)]
final target samples: [(8, 1)]





---------------------------- (4) final statistics
selected variables: [(10, 1), (10, 3), (8, 1)]
omnibus test, n_perm: 500
 -- significant

sequential maximum statistic, n_perm: 500
final source samples: [(8, 1)]
final target samples: [(10, 1), (10, 3)]



Target: 11 - testing sources [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

---------------------------- (1) include target candidates
candidate set: [(11, 1), (11, 2), (11, 3), (11, 4), (11, 5)]
testing candidate: (11, 1) maximum statistic, n_perm: 200
testing candidate: (11, 4) maximum statistic, n_perm: 200
 -- not significant

---------------------------- (3) prune source candidate
selected candidates: [(10, 2)]
 -- significant

---------------------------- (4) final statistics
selected variables: [(11, 1), (11, 2), (11, 5), (10, 2)]
omnibus test, n_perm: 500
testing candidate: (11, 2) maximum statistic, n_perm: 200
 -- significant

sequential maximum statistic, n_perm: 500
testing candidate: (11, 3) maximum statistic, n_perm: 200
final sou

KeyError: 'mehtods'